In [8]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from PIL import Image

ModuleNotFoundError: No module named 'Image'

In [20]:
# Load in array of 201 images, create dataframe
phi = np.load("Phi_201snap.npy")
df = pd.DataFrame(index=np.arange(201))
my_dpi = 192

r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

for i in range(201):
    arr = phi[:,:,i]
    values = arr.flatten()
    
    # Interpolate the data to the Cartesian grid
    grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')

    # Create convert function to use to convert im_array (203x203) back to raw values
    phimax, phimin = np.max(grid_z), np.min(grid_z)
    ratio = (phimax-phimin)/255
    def convertRGBToRaw(x):
        return ratio*x + phimin
    
    # Plot and save image as PNG and JPG, reopen JPG as np array
    plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
    plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
    plt.axis("off")
    plt.savefig('2.2.png', bbox_inches='tight', pad_inches=0)
    
    for q in [100, 75, 50, 25]:
        title = '2.2.' + str(q) + '.jpg'
        plt.savefig(title, bbox_inches='tight', pad_inches=0, dpi=(q*my_dpi/100))
        im = Image.open(title)
        im = im.resize((203, 203), Image.LANCZOS)
        im_array = np.asarray(im)
    
        # Loop through im_array and replace RGB by raw in copy (203x203)
        copy = grid_z.copy()
        for row_index, row in enumerate(im_array):
            for col_index, col in enumerate(row):
                copy[row_index][col_index] = convertRGBToRaw(col[0])
    
        # Get output_array from copy by computing nearest pixels
        output_array = np.zeros((101, 227))
        for rad in range(101):
            for ang in range(227):
                # Polar coordinates
                radius = r[rad]
                angle = theta[ang]
                
                # Convert to Cartesian coordinates
                x_cart = radius * np.cos(angle)
                y_cart = radius * np.sin(angle)
                
                # Find the nearest pixel in the Cartesian grid
                x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
                y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
                
                # Get the raw values from copy and put into output array
                output_array[rad, ang] = copy[y_nearest, x_nearest]
                
        # Compute compression ratio and RMSE's, add to dataframe
        in_imsize = os.stat('2.2.png').st_size
        out_imsize = os.stat(title).st_size
        crcol = 'CR2.2.' + str(q)
        df.at[i, crcol] = in_imsize/out_imsize
        
        diff_array = output_array - arr
        rmse = np.sqrt(np.mean(diff_array**2))
        rmsecol = 'RMSE2.2.' + str(q)
        df.at[i, rmsecol] = rmse
    plt.close()
    
df.to_pickle("analysis2.2.pkl")
df

,CR2.2.100,RMSE2.2.100,CR2.2.75,RMSE2.2.75,CR2.2.50,RMSE2.2.50,CR2.2.25,RMSE2.2.25
0,5.593847,8.196205e-11,7.568493,9.770478e-11,10.601476,1.328808e-10,15.837927,2.463017e-10
1,5.970850,6.527043e-11,8.283064,7.553653e-11,11.844218,1.066376e-10,17.766327,2.059812e-10
2,5.481357,4.644386e-11,7.444343,5.622364e-11,10.630619,7.753508e-11,16.516194,1.480423e-10
3,5.962810,4.618577e-11,8.127816,5.425235e-11,11.666045,7.548834e-11,18.194956,1.412555e-10
4,5.937298,4.901204e-11,8.146341,5.898633e-11,11.708094,8.079382e-11,18.116371,1.499082e-10
...,...,...,...,...,...,...,...,...
196,5.910355,1.034624e-02,7.921284,1.216158e-02,11.132460,1.660012e-02,16.644178,2.938850e-02
197,5.838121,1.133318e-02,7.853264,1.338165e-02,10.985391,1.805124e-02,16.472070,3.216867e-02
198,5.736336,1.258253e-02,7.721762,1.476869e-02,10.830669,1.995737e-02,16.198913,3.546910e-02
199,5.766874,1.402025e-02,7.715101,1.649258e-02,10.907557,2.241504e-02,16.301535,3.997027e-02


In [5]:
df.isnull().sum().sum()

0

In [26]:
# Testing single images
i = 201

phi = np.load("Phi_201snap.npy")
r = np.linspace(1, 101, 101)
theta = np.linspace(0, 2 * np.pi, 227)
r_grid, theta_grid = np.meshgrid(r, theta, indexing='ij')

# Convert polar coordinates to Cartesian coordinates
x = r_grid * np.cos(theta_grid)
y = r_grid * np.sin(theta_grid)

# Define the target grid for interpolation (203x203 pixels)
grid_x, grid_y = np.linspace(-101, 101, 203), np.linspace(-101, 101, 203)
grid_x, grid_y = np.meshgrid(grid_x, grid_y)

points = np.array([x.flatten(), y.flatten()]).T

arr = phi[:,:,i]
values = arr.flatten()
    
# Interpolate the data to the Cartesian grid
grid_z = griddata(points, values, (grid_x, grid_y), method='nearest')
    
# Plot and save image
plt.figure(figsize=(264/my_dpi, 264/my_dpi), dpi=my_dpi)
plt.imshow(grid_z, extent=(-101, 101, -101, 101), cmap='gray')
plt.axis("off")
plt.savefig('analysis2.2.jpg', bbox_inches='tight', pad_inches=0)
im = Image.open('analysis2.2.jpg')
im_array = np.asarray(im)

# Create convert function to use to convert im_array back to raw values
phimax, phimin = np.max(grid_z), np.min(grid_z)
ratio = (phimax-phimin)/255
def convertRGBToRaw(x):
    return ratio*x + phimin

# Loop through im_array and replace RGB by raw in copy
copy = grid_z.copy()
for i, row in enumerate(im_array):
    for j, col in enumerate(row):
        copy[i][j] = convertRGBToRaw(col[0])

output_array = np.zeros((101, 227))
for rad in range(101):
    for ang in range(227):
        # Polar coordinates
        radius = r[rad]
        angle = theta[ang]
            
        # Convert to Cartesian coordinates
        x_cart = radius * np.cos(angle)
        y_cart = radius * np.sin(angle)
            
        # Find the nearest pixel in the Cartesian grid
        x_nearest = np.argmin(np.abs(grid_x[0] - x_cart))
        y_nearest = np.argmin(np.abs(grid_y[:, 0] - y_cart))
            
        # Get the raw values from copy and put into output array
        output_array[rad, ang] = copy[y_nearest, x_nearest]
        
# Compute norm, add to dataframe
diff_array = output_array - arr
norm = np.linalg.norm(diff_array)
norm

IndexError: index 201 is out of bounds for axis 2 with size 201